前件部のルール部分それぞれに重みを付けルール―の簡略化を目指す

まだ、後件部実数値の閾値は未実装

次回やること：後件部の重みが変化していない

In [34]:
#加工前データ

import xlrd
import pprint
import openpyxl
import numpy as np
from datetime import datetime as dt

start_day = dt(year=2020, month=7, day=30, hour=0)
end_day = dt(year=2021, month=9, day=8, hour=0)
test_start = dt(2021,3,15,0)

day_num = (end_day-start_day).days
total_data_num = day_num+1
test_data_num = (end_day-test_start).days+1
teach_data_num = total_data_num-test_data_num
print(test_data_num)
print(teach_data_num)
print(total_data_num)

data_filename = 'osaka_data_before.xlsx'
save_filename = '3.xlsx'

func_num = 3
x_dimension = 6
total_dimension = func_num**x_dimension
omega_iter = 100
max_iter = omega_iter
l_rate = 0.5

178
228
406


In [35]:
wb = openpyxl.load_workbook('./output/osaka/'+save_filename)
for sheet_name in wb.sheetnames:
    for row in wb[sheet_name]:
        for cell in row:
            #print(cell)
            cell.value = None
wb.save('./output/osaka/'+save_filename)
print("Clean up")

Clean up


In [36]:
#データ関連関数
def get_value_list(t_2d):
    return([[cell.value for cell in row] for row in t_2d])

def get_list_2d(sheet, start_row, end_row, start_col, end_col):
    return get_value_list(sheet.iter_rows(min_row=start_row,
                                          max_row=end_row,
                                          min_col=start_col,
                                         max_col=end_col))

def write_list_2d(sheet, l_2d, start_row, start_col):
    for y, row in enumerate(l_2d):
        for x, cell in enumerate(row):
            sheet.cell(row=start_row + y,
                       column=start_col + x,
                       value=l_2d[y][x])

In [37]:
#データを扱いやすく変形
def set_data(y, Y, data, b, c, omega):
    
    for i in range(total_data_num):
        y.append([(Y[i][0]-np.min(Y))/(np.max(Y)-np.min(Y)),(np.mean(Y)-np.min(Y))/(np.max(Y)-np.min(Y))])
        
    for i in range(total_dimension):
        omega[i] = (i+1)/total_dimension

In [38]:
#メンバーシップ関数
def fun(c, b, fun_data, data):    
    for i in range(total_data_num):
        for j in range(x_dimension):
            for k in range(func_num):
                #print(i,j,data[i][j])
                if data[i][j] < c[k]-b[k]:
                    fun_data[i][k][j] = 0
                if data[i][j] > c[k]+b[k]:
                    fun_data[i][k][j] = 0
                if data[i][j] >= c[k]-b[k] and data[i][j] <= c[k]:
                    fun_data[i][k][j] = 1-abs(c[k]-data[i][j])*(1/b[k])
                if data[i][j] > c[k] and data[i][j] <= c[k]+b[k]:
                    fun_data[i][k][j] = 1-abs(c[k]-data[i][j])*(1/b[k])

In [39]:
#データに関数を適用する関数
def rule(m, total_m, fun_data, memba_num, w, pre_w, omega, num):
    
    if num == total_data_num:
        print("initial")
        for d_num in range(total_data_num):
            for j in range(x_dimension):
                times = 0
                R = func_num**(x_dimension-1-j)
                for p in range(R):
                    for i in range(func_num):
                        r = func_num**j
                        for k in range(r):
                            m[d_num][times] *= fun_data[d_num][i][j] * pre_w[times][j]
                            #memba_num[times][j][i] = 1
                            times+=1
        #学習用とテスト用のμの総和
        #学習用とテスト用のμの総和
        for j in range(total_data_num):
            for i in range(total_dimension):
                total_m[j] += m[j][i] * w[i]
    else:
        d_num = num
        m[d_num].fill(1)
        total_m[d_num] = 0
        
        for j in range(x_dimension):
            times = 0
            R = func_num**(x_dimension-1-j)
            for p in range(R):
                for i in range(func_num):
                    r = func_num**j
                    for k in range(r):
                        m[d_num][times] *= fun_data[d_num][i][j] * pre_w[times][j]
                        times+=1
        
        temp_m = 0
        for M,W,O in zip(m[d_num],w,omega):
            total_m[num] += M * W
            temp_m += M*W*O
        return temp_m/total_m[num]

In [40]:
#最急降下法 改良
def gradient_descent(fun_data, memba_num, total_m, m, y, w, pre_w, omega, mse):
    temp_w = 0
    temp_omega = 0
    
    #後件部の学習
    for i in range(omega_iter):
        #print(str(i)+"回目")
        for j in range(teach_data_num):
            for dim in range(total_dimension):
                if total_m[j] == 0:
                    print("total "+ str(j) +" is 0")
                omega[dim] = omega[dim] - l_rate * (-y[j][0]+y[j][1]) * m[j][dim] / total_m[j]
                w[dim] = w[dim] - l_rate *  (-y[j][0]+y[j][1]) * m[j][dim] * (omega[dim]-y[j][1]) / total_m[j]
                """if m[j][dim] != 1:
                    print(j, dim, m[j][dim])"""
                """for k in range(x_dimension):
                    pre_w[dim][k] -= l_rate *  (-y[j][0]+y[j][1]) * w[dim] * (omega[dim]-y[j][1]) * m[j][dim] / (total_m[j] * pre_w[dim][k])"""
            y[j][1] = rule(m, total_m, fun_data, memba_num, w, pre_w, omega, j)
            
        MES(y, mse, i, w, omega, 'teach', 0)
        if i%10 == 0:
            print(str(i)+"回目")
            #pprint.pprint(y)
            #omega[dim] = temp_omega
            #w[dim] = temp_w
                
    for i in range(omega_iter):
        for j in range(teach_data_num):
            for dim in range(total_dimension):
                if total_m[j] == 0:
                    print("total "+ str(j) +" is 0")
                for k in range(x_dimension):
                    pre_w[dim][k] -= l_rate *  (-y[j][0]+y[j][1]) * w[dim] * (omega[dim]-y[j][1]) * m[j][dim] / (total_m[j] * pre_w[dim][k])
            y[j][1] = rule(m, total_m, fun_data, memba_num, w, pre_w, omega, j)
            
        MES(y, mse, i, w, omega, 'teach', 1)
        if i%10 == 0:
            print(str(i)+"回目")
    
    #重みの学習結果を残しておく
    #時間短縮のため
    save_result(y, w, pre_w, omega, m, total_m, 1)
    
    print("後件部学習終了")
    print("重み学習終了")
    
    #testデータで
    for i in range(teach_data_num,total_data_num):
        y[i][1] = rule(m, total_m, fun_data, memba_num, w, pre_w, omega, i)

    MES(y, mse, max_iter, w, omega, 'test', 0)
    save_result(y, w, pre_w, omega, m, total_m, 2)

In [41]:
def MES(y, mse, n, w, omega, litera, num):
    temp = 0
    
    if litera=='teach':
        for i in range(teach_data_num):
            temp += (y[i][1] - y[i][0])**2
        mse = temp/teach_data_num
    else:
        for i in range(teach_data_num,total_data_num):
            temp += (y[i][1] - y[i][0])**2
        mse = temp/test_data_num
    
    wc = openpyxl.load_workbook('./output/osaka/'+save_filename, data_only=True)
    for sheet_name in wc.sheetnames:
        if sheet_name == 'output':
            wc["sq_error"].cell(column=1+num, row=1+n, value = mse)
            
            wc.save('./output/osaka/'+save_filename) 
        else:
            """err_sheet = wc.create_sheet('sq_error')
            err_sheet.cell(column=1, row=1+n, value = mse)
            wc.save('./output/osaka/'+save_filename) """

In [42]:
#excelの保存
def save_result(output_list, w, pre_w, omega, m, total_m, num):
    wc = openpyxl.load_workbook('./output/osaka/'+save_filename, data_only=True)
    for sheet_name in wc.sheetnames:
        if sheet_name == 'output':
            if num == 1:
                omega_sheet = wc["omega"]
                for i in range(total_dimension):
                    omega_sheet.cell(column=1, row=i+1, value = omega[i])

                w_sheet = wc["w"]
                for i in range(total_dimension):
                    w_sheet.cell(column=1, row=i+1, value = w[i])
                    
                prew_sheet = wc["pre_w"]
                for i in range(total_dimension):
                    for j in range(x_dimension):
                        prew_sheet.cell(column=1+j, row=i+1, value = pre_w[i][j])

                m_sheet = wc["m"]
                for i in range(total_data_num):
                    for j in range(total_dimension):
                        m_sheet.cell(column=1+i, row=1+j, value = m[i][j])

                total_m_sheet = wc["total_m"]
                for i in range(total_data_num):
                    total_m_sheet.cell(column=1+i, row=1, value = total_m[i])
            else:
            
                write_list_2d(wc["output"],[['理想値','出力値']], 1, 1)
                write_list_2d(wc["output"],output_list, 2, 1)

                #wc["sq_error"].cell(column=1, row=1, value = mse)

            wc.save('./output/osaka/'+save_filename)
            print("Already exist. Save complited ")  
            return 

In [43]:
#出力yを導出する関数
def make_output(total_m, m, y, fun_data, memba_num, data, omega, c, b, w, pre_w, sq_err):
    
    print("開始")
    #学習
    gradient_descent(fun_data, memba_num, total_m, m, y, w, pre_w, omega, sq_err)
    print("完了")
    
    return 0

In [44]:
from sklearn import preprocessing
import random

def main():
    #Excelの指定
    wb = openpyxl.load_workbook('./data/'+data_filename, data_only=True)
    data_sheet = wb["osaka_data_before"]

    '''
    data：データ
    y：病床占有率
    fun_data：関数を通したデータの値
    m：ルールに対する適合度
    total_m：適合度の総和
    omega：ルール後件部
    pre_w:ルール前件部の各重み
    w：ルールの重み
    mse：平均2乗誤差
    c：関数の中心
    b：関数の幅
    '''
    random.seed(0)
    
    start_row = 2
    data = get_list_2d(data_sheet, start_row, total_data_num+start_row-1, 2, x_dimension+1)
    Y = get_list_2d(data_sheet, start_row, total_data_num+start_row-1, x_dimension+2, x_dimension+2)
    y = []
    fun_data = np.empty((total_data_num, func_num, x_dimension))
    m = np.ones((total_data_num, total_dimension))
    total_m = np.zeros(total_data_num)
    omega = np.zeros(total_dimension)
    pre_w = np.random.uniform(0.8,1,(total_dimension, x_dimension))
    w = np.ones(total_dimension)
    mse = np.zeros(max_iter)
    #適合度の計算の際にどのメンバシップ値をかけているのかを記憶する
    memba_num = np.zeros((total_dimension, x_dimension, func_num))
    
    #中心と幅の初期値の設定
    c = [0, 0.5, 1] 
    b = [1, 0.5, 1] #変えるべき
    
    #参照問題でここで正規化
    #保存して、読み込みでもいいかも
    mm = preprocessing.MinMaxScaler()
    data = mm.fit_transform(data)
    print("正規化完了")
    
    #データのセット
    set_data(y, Y, data, b, c, omega)
    print("データセット完了")
    
    fun(c, b, fun_data, data)
    rule(m, total_m, fun_data, memba_num, w, pre_w, omega, total_data_num)

    make_output(total_m, m, y, fun_data, memba_num, data, omega, c, b, w, pre_w, mse)
    
    wc = openpyxl.load_workbook('./output/osaka/'+save_filename, data_only=True)
    inversed = np.multiply(y, np.max(Y)-np.min(Y))+np.min(Y)
    write_list_2d(wc["fix"],[['理想値','出力値']], 1, 1)
    write_list_2d(wc["fix"],inversed, 2, 1)
    wc.save('./output/osaka/'+save_filename)

if __name__ == '__main__':
    
    main()

正規化完了
データセット完了
initial
開始
0回目
10回目
20回目
30回目
40回目
50回目
60回目
70回目
80回目
90回目
0回目
10回目
20回目
30回目
40回目
50回目
60回目
70回目
80回目
90回目
Already exist. Save complited 
後件部学習終了
重み学習終了
Already exist. Save complited 
完了
